In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import pickle
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras.layers import Dense, Dropout
import tensorflow_hub as hub
import tensorflow_text as text

In [ ]:
df=pd.read_csv("/kaggle/input/mcdonalds-store-reviews/McDonald_s_Reviews.csv",encoding='latin-1')
df.head(5)

In [ ]:
df.info()

In [ ]:
rat=df.rating.value_counts()
rat

In [ ]:

import matplotlib.pyplot as plt

In [ ]:
fig, ax = plt.subplots()
ax.bar(rat.index, rat.values)

# Set labels and title
ax.set_xlabel('Rating')
ax.set_ylabel('Values')
ax.set_title('Rating Value Count')

# Display the plot
plt.show()


In [ ]:
import re
df["rating"] = df["rating"].str.extract(r"(\d+)", expand=False).astype(int)

In [ ]:
mapping={1:0,2:0,3:None,4:1,5:1}
df.rating=df.rating.map(mapping)
df=df.dropna(subset='rating')

In [ ]:
df["review"] = df["review"].str.encode("ascii", "ignore").str.decode("utf-8")

In [ ]:
df = df[["review", "rating"]]

In [ ]:
df.head(5)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(df.review,df.rating,test_size=0.2)

In [ ]:
X_train.shape

In [ ]:
X_test.shape

In [ ]:
bert_preprocess = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3")
bert_encoder = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/4")

In [ ]:
def get_sentense_embeding(sentences):
    preprocesed_text=bert_preprocess(sentences)
    return bert_encoder(preprocesed_text)['pooled_output']
get_sentense_embeding([
    df.review[5],df.review[1]
])    

In [ ]:
text_input=tf.keras.layers.Input(shape=(),dtype=tf.string,name='text')
preprocessed_text=bert_preprocess(text_input)
outputs=bert_encoder(preprocessed_text)
l=tf.keras.layers.Dropout(0.1,name='dropout')(outputs['pooled_output'])
l=tf.keras.layers.Dense(1,activation='sigmoid',name='output')(l)

model=tf.keras.Model(inputs=[text_input],outputs=[l])

In [ ]:
model.summary()

In [ ]:
METRICS=[
    tf.keras.metrics.BinaryAccuracy(name='accuracy'),
    tf.keras.metrics.Precision(name='precision'),
    tf.keras.metrics.Recall(name='recall')
]
model.compile(optimizer='adam',loss='binary_crossentropy',metrics=METRICS)

In [ ]:
model.fit(X_train,y_train,epochs=10)

In [ ]:
model.evaluate(X_test,y_test)

In [ ]:
y_pred=model.predict(X_test)
y_pred=y_pred.flatten()

In [ ]:
y_pred=np.where(y_pred>0.5,1,0)
y_pred

In [ ]:
from sklearn.metrics import confusion_matrix, classification_report
cm=confusion_matrix(y_test,y_pred)

In [ ]:

from matplotlib import pyplot as plt
import seaborn as sns
sns.heatmap(cm,annot=True,fmt='d')

plt.xlabel("Predicted")
plt.ylabel("Truth")

In [ ]:
print(classification_report(y_test,y_pred))

In [ ]:
reviews=[df.review[0],df.review[1]
    
]
model.predict(reviews)